# How to get Comments

In [3]:
import pandas as pd

In [4]:
def Get_Comments(PostID):
    df = []
    break_times = 0
    fail_cnt = 1
    offset=0
    url = 'https://www.facebook.com/ajax/ufi/comment_fetch.php'
    while fail_cnt <= 30:
        try:
            params = {'ft_ent_identifier': PostID,
                      'offset': offset,
                      'length': 50,
                      '__a': 1}
            resp = requests.get(url, params=params)
            data = json.loads(re.sub(r'for \(;;\);','',resp.text))
            id2name=pd.DataFrame(pd.DataFrame(data['jsmods']['require'])[3][0][1]['profiles']).loc[['name','gender'],:].T.reset_index()
            id2name.columns = ['AUTHOR', 'NAME','GENDER']
            replies = pd.DataFrame(pd.DataFrame(data['jsmods']['require'])[3][0][1]['commentlists']['replies']).loc['count',:].reset_index()
            replies['index'] = replies['index'].apply(lambda x: x.split('_', -1)[1])
            replies.columns = ['COMMENTID', 'REPLIES']

            df2 = []
            for i in pd.DataFrame(data['jsmods']['require'])[3][0][1]['comments']:
                ndf = pd.DataFrame(data = [{'TYPENAME':i['__typename'],
                                            'AUTHOR':i['author'],
                                            'TEXT':i['body']['text'],
                                            'POSTID': re.findall('[0-9]{1,}', i['id'])[0],
                                            'COMMENTID': re.findall('[0-9]{1,}', i['id'])[1],
                                            'PARENTCOMMENTAUTHORID': i['parentcommentauthorid'],
                                            'PARENTCOMMENTID': i['parentcommentid'],
                                            'LIKECOUNT': i['likecount'],
                                            'TIME':datetime.datetime.fromtimestamp(int(i['timestamp']['time'])).strftime("%Y-%m-%d %H:%M"),
                                            'UPDATETIME':datetime.datetime.now().strftime("%Y-%m-%d %H:%M")}],
                                   columns=['AUTHOR', 'TIME' ,'TEXT','TYPENAME','COMMENTID','PARENTCOMMENTAUTHORID', 'PARENTCOMMENTID', 'LIKECOUNT', 'POSTID', 'UPDATETIME'])
                df2.append(ndf)
            # 合併資料
            df2 = pd.concat(df2, ignore_index=True)
            df2 = pd.merge(df2, id2name,  how='left', on='AUTHOR')
            df2 = pd.merge(df2, replies,  how='left', on='COMMENTID')
            df.append(df2)
            break_times = 0
            fail_cnt = 0
            
        except:
            fail_cnt += 1

        offset += 50
    # 完成後更新
    df = pd.concat(df,ignore_index=True)
    df = df.loc[:,['NAME', 'AUTHOR', 'TIME', 'TEXT', 'REPLIES', 'LIKECOUNT', 'POSTID', 'COMMENTID', 'PARENTCOMMENTAUTHORID', 'PARENTCOMMENTID', 'GENDER', 'TYPENAME', 'UPDATETIME']]
    print(df.shape)
    return df

In [5]:
# %%time
df = Get_Comments('160361789041991')
df

ValueError: No objects to concatenate

In [ ]:
url = 'https://www.facebook.com/api/graphql/'
headers = {'accept-language': 'en',
           'content-length': '2393',
           'content-type': 'application/x-www-form-urlencoded',
           'cookie': 'sb=GIJAYhYqNFMgnfrPOBOMrVTD; dpr=1.190000057220459; datr=a4JAYqZ-X0No1tC9N8Yt90_n; fr=0HWWm43K042LlVfPQ..BiQIIY.at.AAA.0.0.BiQIKC.AWWD5NBTIqY; wd=881x852',
           'dnt': '1',
           'origin': 'https://www.facebook.com',
           'referer': 'https://www.facebook.com/BToday/',
           'sec-ch-prefers-color-scheme': 'dark',
           'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Microsoft Edge";v="99"',
           'sec-ch-ua-mobile': '?0',
           'sec-ch-ua-platform': "Windows",
           'sec-fetch-dest': 'empty',
           'sec-fetch-mode': 'cors',
           'sec-fetch-site': 'same-origin',
           'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36 Edg/99.0.1150.55',
           'x-fb-friendly-name': 'CometUFICommentsProviderQuery'}
data = {'av': '0',
        '__user': '0',
        '__a': '1',
        '__dyn': '7xeUmwlEnwn8K2WnFw9-2i5U4e0yoW3q322aew9G2S0zU2lwUx60gu0luq1ew65xO0FE2awgolzUO0-E4a3aUS2G3i0Boy1PwBgao6C0Mo5W3S1lwlE-UqwsUkxe2Gew9O222SUbEaU2eUlDw-waCm7-8wSweK2S3qazo3iwPwbS16AzUjw',
        '__csr': 'grNc8N-xif9HlGzli9ZIAzGAFqBK-GiSh2QFaqiGrGFUOZ5ykFWQqmiBybCBgWuqpeKlAKLtV-ibyWJUBeiqcyd1WWGewBxKF9UiyppQiayKu8Cx2l3UScF1mu8z8lUF5yo4u7awsGxe10wa27bw9q0rq2fySq9w5uw4yw2j80Hq5o1Uiw56zd120uF09Kq0n2fUeO05kzE2EwvE14UDy8y4uiE70K7UCUSZlxy8Lg1gmu1Cwfa1Sw13m6e0mu4rggsN80zm0Co1480v8w0Xiw3qEkCx21xK5E0mmw4MwuE1wovwdi3q0Y8bQ0O8WgyyGy9Ul83HWK11G4UCFbF27o2GnwDkHwBx2y0a6--h_cXz8Oh6-A5qAGZkzqg1LKV20LS1jjleq9Hbh-O4BAButvAFb-RJ5BUe9kaQjhVGwn40i-09nwYw5bVku0PF4fK12BwCwwAxq5825wFwgo6SdwJyUd8bEcEioeU1kU4W1yy18C2a2pdEM1hQdw8G2G0w88orwlUW4EqGu0HE2uCxy6BgeoO0Ie268wRwaO0YQ0hq681CU2nxHo4W1bwe20sGczkUrAUCAdxy740w86C1_Bl0fe0imm2t0vUpCw9O0IejBGtQ1ew42wyg2rK0xWBz5QA2NehmIxUmXG6U0EK1Tyo0I_wXwcpe1-w2D-bwLCwdV0ok8wCGOlqgVVExmil0aN0mo25wDoa9AdwlE1NV-Nw2Aw5sw8K0EFFVe0QaglwdS04EElnwHwj81pC',
        '__req': 'v',
        '__hs': '19078.HYP:comet_loggedout_pkg.2.0.0.0.',
        'dpr': '1.5',
        '__ccg': 'EXCELLENT',
        '__rev': '1005252446',
        '__s': 'ccit5s:x56tk6:tkfbk8',
        '__hsi': '7079802625766252367-0',
        '__comet_req': '1',
        'lsd': 'AVrqrndyyI0',
        'jazoest': '21065',
        '__spin_r': '1005252446',
        '__spin_b': 'trunk',
        '__spin_t': '1648395002',
        'fb_api_caller_class': 'RelayModern',
        'fb_api_req_friendly_name': 'CometUFICommentsProviderQuery',
        'variables': str({"UFI2CommentsProvider_commentsKey":"CometSinglePageContentContainerFeedQuery",
                          "__false":false,
                          "__true":true,
                          "after":"AQHRQvG8VVb_Ra8-s7lZO_OMhymo5sTm-CUUq9J7F8ApUlEgOHqnAiwYYWLxlZGn9FhEQeANLdqSl9aQWsNfcy55lQ",
                          "before":null,
                          "displayCommentsContextEnableComment":null,
                          "displayCommentsContextIsAdPreview":null,
                          "displayCommentsContextIsAggregatedShare":null,
                          "displayCommentsContextIsStorySet":null,
                          "displayCommentsFeedbackContext":null,
                          "feedLocation":"PAGE_TIMELINE",
                          "feedbackSource":22,
                          "first":44,
                          "focusCommentID":null,
                          "includeHighlightedComments":false,
                          "includeNestedComments":true,
                          "initialViewOption":"RANKED_THREADED",
                          "isInitialFetch":false,
                          "isPaginating":true,
                          "last":null,
                          "scale":1.5,
                          "topLevelViewOption":null,
                          "useDefaultActor":false,
                          "viewOption":null,
                          "id":"ZmVlZGJhY2s6MTAxNTk5MTM1NTIzOTMyNzA="}'}
                         